In [ ]:
import cv2
import numpy as np
import os

In [ ]:
# Yolov4 yapılandırma ve ağırlıkları
config_path = "yolov3-tiny.cfg"
weights_path = "yolov3-tiny-prn.weights"
classes_path = "yolo11l-seg.txt"

image_path   = "car.jpg"

In [ ]:
print("Current Working Directory:", os.getcwd())
print("Prototxt Dosyası Var mı?:", os.path.exists(config_path))
print("Model Dosyası Var mı?:", os.path.exists(weights_path))
print("class Dosyası Var mı?:", os.path.exists(classes_path))
print("img Dosyası Var mı?:", os.path.exists(image_path))

In [ ]:
# Sınıf isimlerini oku
with open(classes_path, "r") as f:
    class_names = [line.strip() for line in f.readlines()]



In [ ]:
print(class_names)

In [ ]:
# Ağ modeli yükle
net = cv2.dnn.readNetFromDarknet(config_path, weights_path)


net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)  # CPU kullanılır
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)



In [ ]:
# Resmi oku
img = cv2.imread(image_path)
height, width = img.shape[:2]



In [ ]:
# Girdi için blob hazırla
blob = cv2.dnn.blobFromImage(img, 1/255.0, (416, 416), swapRB=True, crop=False)
net.setInput(blob)



In [ ]:
# Çıkış katmanlarının adını al
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers().flatten()]



In [ ]:
# İleri yayılım (forward)
outputs = net.forward(output_layers)



In [ ]:
# Tespitleri işle
conf_threshold = 0.5
nms_threshold = 0.4
boxes, confidences, class_ids = [], [], []



In [ ]:
for output in outputs:
    for detection in output:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > conf_threshold:
            center_x, center_y, w, h = (detection[0:4] * [width, height, width, height]).astype("int")
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)
            boxes.append([x, y, int(w), int(h)])
            confidences.append(float(confidence))
            class_ids.append(class_id)



In [ ]:
# NMS (Gereksiz kutuları filtreleme)
indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)

for i in range(len(indices)):
    i = indices[i][0] if isinstance(indices[i], (list, tuple, np.ndarray)) else indices[i]
    x, y, w, h = boxes[i]
    label = f"{class_names[class_ids[i]]}: {confidences[i]:.2f}"
    cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
    cv2.putText(img, label, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

In [ ]:
# Sonucu göster
cv2.imshow("YOLOv3-tiny Detection", img)
cv2.waitKey(0)
cv2.destroyAllWindows()
